In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import date, timedelta
from sklearn.cluster import KMeans

https://www.kaggle.com/vanshjatana/analysis-on-coronavirus
이곳을 참조 했습니다.

In [ ]:
path = '/kaggle/input/coronavirusdataset/'
patient_data_path = path + 'patient.csv'
route_data_path = path + 'route.csv'
time_data_path = path + 'time.csv'

patient = pd.read_csv(patient_data_path)
route = pd.read_csv(route_data_path)
time = pd.read_csv(time_data_path)

변수확인
1.id the ID of the patient (n-th confirmed patient)
2.sex the sex of the patient
3.birth_year the birth year of the patient
4.country the country of the patient
5.region the region of the patient
6.group the collective infection
7.infection_reason the reason of infection
8.infection_order the order of infection
9.infected_by the ID of who has infected the patient
10.contact_number the number of contacts with people
11.confirmed_date the date of confirmation
12.released_date the date of discharge
13.deceased_date the date of decease
14.state isolated / released / deceased

In [ ]:
patient.head()

In [ ]:
time.head()

In [ ]:
route.head()

결측치 확인

In [ ]:
patient.isna().sum()

In [ ]:
patient['birth_year'] = patient.birth_year.fillna(0.0).astype(int)
patient['birth_year'] = patient['birth_year'].map(lambda val: val if val > 0 else np.nan)

전처리

In [ ]:
date_cols = ["confirmed_date", "released_date", "deceased_date"]
for col in date_cols:
    patient[col] = pd.to_datetime(patient[col])

In [ ]:
patient["time_to_release_since_confirmed"] = patient["released_date"] - patient["confirmed_date"]
patient["time_to_death_since_confirmed"] = patient["deceased_date"] - patient["confirmed_date"]
patient["duration_since_confirmed"] = patient[["time_to_release_since_confirmed", "time_to_death_since_confirmed"]].min(axis=1)
patient["duration_days"] = patient["duration_since_confirmed"].dt.days

Accumulated Confirmed Count

In [ ]:
patient.confirmed_date = pd.to_datetime(patient.confirmed_date)
daily_count = patient.groupby(patient.confirmed_date).id.count()
accumulated_count = daily_count.cumsum()

In [ ]:
accumulated_count.plot()
plt.title('Accumulated Confirmed Count');

Accumulated Relaesed Count

In [ ]:
patient.released_date = pd.to_datetime(patient.released_date)
daily_count_released = patient.groupby(patient.released_date).id.count()
accumulated_count_released = daily_count_released.cumsum()
accumulated_count_released.plot()
plt.title('Accumulated Relaesed Count');

Accumulated deceased Count

In [ ]:
patient.deceased_date = pd.to_datetime(patient.deceased_date)
daily_count_deceased = patient.groupby(patient.deceased_date).id.count()
accumulated_count_deceased = daily_count_deceased.cumsum()
accumulated_count_deceased.plot()
plt.title('Accumulated deceased Count');

In [ ]:
accumulated_count_deceased.plot(label="deceased")
accumulated_count_released.plot(label="released")
plt.legend(loc=2)

In [ ]:
accumulated_count_deceased.plot(label="deceased")
accumulated_count_released.plot(label="released")
accumulated_count.plot(label="Confirmed")
plt.legend(loc=2)

Age distribution of death

In [ ]:
patient['age'] = 2020 - patient['birth_year']

In [ ]:
dead = patient[patient.state == 'deceased']

In [ ]:
dead["age"].describe()


In [ ]:
plt.figure(figsize=(10,6))
sns.set_style("darkgrid")
plt.title("Age distribution of the deceased")
sns.kdeplot(data=dead['age'], shade=True)

Age distribution of released

In [ ]:
released = patient[patient.state == 'released']

In [ ]:
released['age']. describe()

In [ ]:
plt.figure(figsize=(10,6))
sns.set_style("darkgrid")
plt.title("Age distribution of the released")
sns.kdeplot(data=released['age'], shade=True)

Comparison of released and deceased by age

In [ ]:
sns.kdeplot(data=dead['age'],label='deceased', shade=True)
sns.kdeplot(data=released['age'],label='released', shade=True)

Age distribution of death by gender

In [ ]:
male_dead = dead[dead.sex=='male']
female_dead = dead[dead.sex=='female']

In [ ]:
plt.figure(figsize=(10,6))
sns.set_style(("ticks"))
plt.title("Age distribution of the deceased by gender")
sns.kdeplot(data=female_dead['age'],label='femele', shade=True)
sns.kdeplot(data=male_dead['age'],label='mele', shade=True)

Reason for the infection

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Infection reason')
patient.infection_reason.value_counts().plot.bar()

Patient in province

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Number patients in province')
route.province.value_counts().plot.bar()

patients in group

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Number patients in group')
patient.group.value_counts().plot.bar()

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Number patients in infection_reason')
patient.infection_reason.value_counts().plot.bar()

Released by age

In [ ]:
plt.figure(figsize=(25,5))
sns.barplot(
    data= patient,
    x= "age",
    y= "duration_days"
)
plt.title('released by age')